# ИД23-1 Маслов АН

## Функция нормализации БД и сравнение sqlite3 с pandas
### Вариант supermarket_sales -.csv

In [5]:
def databaseNormalize(original_df, con, cur, columns):
    """
    Функция нормализует переданный DataFrame, c учетом переданных столбцов и создает необходимые таблицы в базе данных.
    con и cur - соединение и курсор БД
    """
    # Создаем таблицы в БД с уникальными значениями
    for i in columns:
        unique_array = original_df[i].unique()
        cur.execute(f'''CREATE TABLE IF NOT EXISTS "{i}_table" (ID INTEGER PRIMARY KEY, "{i}" TEXT UNIQUE)''')
        try: 
            cur.executemany(f'''INSERT INTO "{i}_table" ("{i}") VALUES (?)''', [(j,) for j in unique_array])
        except:
            print (f"Добавляемые данные уже внесены в {i}_table!")
        con.commit()
    # Создаем таблицу в БД для переданного DataFrame
    cur.execute('''CREATE TABLE IF NOT EXISTS "supermarket_sales_table" (
            "Invoice ID" TEXT PRIMARY KEY,
            "Branch" TEXT,
            "City" INTEGER,
            "Customer type" TEXT,
            "Gender" TEXT,
            "Product line" INTEGER,
            "Unit price" DOUBLE,
            "Quantity" INTEGER,
            "Tax 5%" DOUBLE,
            "Total" DOUBLE,
            "Date" TEXT,
            "Time" TEXT,
            "Payment" INTEGER,
            "Cogs" DOUBLE,
            "Gross margin percentage" DOUBLE,
            "Gross income" DOUBLE,
            "Rating" DOUBLE,
        
            CONSTRAINT fk_city FOREIGN KEY ("City") REFERENCES "City_table"(ID),
            CONSTRAINT fk_product_line FOREIGN KEY ("Product line") REFERENCES "Product line_table"(ID),
            CONSTRAINT fk_payment FOREIGN KEY ("Payment") REFERENCES "Payment_table"(ID)
            )''')
    # Нормализуем основную таблицу в БД
    for index, row in original_df.iterrows():
        for i in columns:
            cur.execute(f'''SELECT ID FROM "{i}_table" WHERE "{i}" = ?''', (row[i],))
            id_value = cur.fetchone()[0]
            original_df.at[index, i] = id_value
    # Нормализуем DataFrame
    try: 
        original_df.to_sql("supermarket_sales_table", con, if_exists='append', index=False)
    except: 
        print("Добавляемые данные уже внесены в supermarket_sales_table!")
    return original_df

In [6]:
import pandas as pd
import sqlite3 as sql
import time
# Устанавливаем соединение с БД
con = sql.connect('supermarket.db')
# Создаем объект-курсор
cur = con.cursor()
original_df = pd.read_csv("supermarket_sales -.csv")
unique_columns = ['City', 'Product line', 'Payment']
original_df =  databaseNormalize(original_df, con, cur, unique_columns)
# Создаем DataFrame для каждой оставшийся таблицы
city_df = pd.read_sql("SELECT * FROM 'City_table'", con)
product_df = pd.read_sql("SELECT * FROM 'Product line_table'", con)
payment_df = pd.read_sql(" SELECT * FROM 'Payment_table'", con)
# Сверяем размеры таблицы БД и DataFrame
cur.execute("SELECT COUNT(*) FROM supermarket_sales_table ")
print(f"Количество строк в основной таблице в БД:", cur.fetchone()[0])
print(f"Количество строк в основном DataFrame:", original_df.shape[0])



Количество строк в основной таблице в БД: 1000
Количество строк в основном DataFrame: 1000


In [7]:
timer_start = time.time()
# Выполняет сложный запрос, который соединяет таблицы на основе внешних ключей, подсчитывает количество продаж в
# каждом городе по каждому типу продукции и типу платежей, и группирует результат 
cur.execute("""
            SELECT c.City, p.'Product line', pay.Payment, COUNT(*) AS Number_of_Sales
            FROM supermarket_sales_table AS s 
            LEFT JOIN City_table AS c ON s.City = c.ID 
            LEFT JOIN 'Product line_table' AS p ON s.'Product line' = p.ID 
            LEFT JOIN Payment_table AS pay ON s.Payment = pay.ID 
            GROUP BY c.City, p."Product line", pay.Payment
        """)
print([i[0] for i in cur.description])
# Вывод результата запроса
for v in cur: 
    print(v)    
timer_end = time.time()
print(f"\nСкорость работы сложного запроса через sqlite3:", timer_end - timer_start)

['City', 'Product line', 'Payment', 'Number_of_Sales']
('Mandalay', 'Electronic accessories', 'Cash', 24)
('Mandalay', 'Electronic accessories', 'Credit card', 15)
('Mandalay', 'Electronic accessories', 'Ewallet', 16)
('Mandalay', 'Fashion accessories', 'Cash', 22)
('Mandalay', 'Fashion accessories', 'Credit card', 19)
('Mandalay', 'Fashion accessories', 'Ewallet', 21)
('Mandalay', 'Food and beverages', 'Cash', 8)
('Mandalay', 'Food and beverages', 'Credit card', 25)
('Mandalay', 'Food and beverages', 'Ewallet', 17)
('Mandalay', 'Health and beauty', 'Cash', 17)
('Mandalay', 'Health and beauty', 'Credit card', 17)
('Mandalay', 'Health and beauty', 'Ewallet', 19)
('Mandalay', 'Home and lifestyle', 'Cash', 13)
('Mandalay', 'Home and lifestyle', 'Credit card', 15)
('Mandalay', 'Home and lifestyle', 'Ewallet', 22)
('Mandalay', 'Sports and travel', 'Cash', 26)
('Mandalay', 'Sports and travel', 'Credit card', 18)
('Mandalay', 'Sports and travel', 'Ewallet', 18)
('Naypyitaw', 'Electronic acces

In [8]:

timer_start = time.time()
# Выполняет запрос из прошлой ячейки, но через функции Pandas
# Объединяем DataFrame
merged = (
    original_df.merge(city_df, left_on="City", right_on="ID", how="left")
     .merge(product_df, left_on="Product line", right_on="ID", how="left")
     .merge(payment_df, left_on="Payment", right_on="ID", how="left")
)
# Группируем и считаем количество продаж и сумму
result = (
    merged.groupby(["City_y", "Product line_y", "Payment_y"], as_index=False)
          .agg(Number_of_Sales=("Total", "count"))   # количество продаж

)
# Переименовываем столбцы для соответствия выводу SQL-запроса
result.rename(columns={
    "City_y": "City",
    "Product line_y": "Product line",
    "Payment_y": "Payment"
}, inplace=True)
print(result)
timer_end = time.time()
print(f"\nСкорость работы сложного запроса через pandas:", timer_end - timer_start)
# Закрываем соединение
cur.close()
con.close()

         City            Product line      Payment  Number_of_Sales
0    Mandalay  Electronic accessories         Cash               24
1    Mandalay  Electronic accessories  Credit card               15
2    Mandalay  Electronic accessories      Ewallet               16
3    Mandalay     Fashion accessories         Cash               22
4    Mandalay     Fashion accessories  Credit card               19
5    Mandalay     Fashion accessories      Ewallet               21
6    Mandalay      Food and beverages         Cash                8
7    Mandalay      Food and beverages  Credit card               25
8    Mandalay      Food and beverages      Ewallet               17
9    Mandalay       Health and beauty         Cash               17
10   Mandalay       Health and beauty  Credit card               17
11   Mandalay       Health and beauty      Ewallet               19
12   Mandalay      Home and lifestyle         Cash               13
13   Mandalay      Home and lifestyle  Credit ca